In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from matplotlib import pyplot as plt

from micro_price_trading import Preprocess, TwoAssetSimulation
from micro_price_trading.config import TWENTY_SECOND_DAY, DATA_PATH

In [2]:
with open('simulations.npy', 'rb') as file:
    input_data = np.load(file)

In [3]:
raw = Preprocess('Test_TBT_TBF_data.csv', res_bin = 7)
data = raw.process()

In [4]:
def CI(array, confidence = .95):
    half_width = 1.96*array.std()/np.sqrt(len(array))
    mu = array.mean()
    return mu - half_width, mu, mu + half_width

In [5]:
def base_twap(t, asset):
    
    indices_to_buy_at = np.array([idx for idx in range(t, TWENTY_SECOND_DAY+1, t)])
    asset_to_buy = asset
    
    avg_prices = input_data[:,indices_to_buy_at, asset_to_buy].mean(axis = 1)
    
    return avg_prices

In [6]:
CI(base_twap(5, 1)), CI(base_twap(5, 2))

((18.312610608531543, 18.31477538888889, 18.316940169246237),
 (16.660154842865417, 16.661335094017097, 16.662515345168778))

In [31]:
def random_twap(t, weights=(0.5, 0.5)):
    
    indices_to_buy_at = np.array([idx for idx in range(t, TWENTY_SECOND_DAY+1, t)])
    random_numbers = np.random.rand(len(indices_to_buy_at))

    indices_to_buy_asset_1 = indices_to_buy_at[np.where(random_numbers >= weights[0])[0]]
    indices_to_buy_asset_2 = indices_to_buy_at[np.where(random_numbers < weights[0])[0]]

    avg_prices_1 = input_data[:,indices_to_buy_asset_1, 1].mean(axis = 1)
    avg_prices_2 = input_data[:,indices_to_buy_asset_2, 2].mean(axis = 1)
    
    return avg_prices_1, avg_prices_2

In [32]:
CI(random_twap(10, weights=(0.5, 0.5))[0]), CI(random_twap(10, weights=(0.5, 0.5))[1])

((18.28824100061176, 18.33005263157896, 18.371864262546158),
 (16.626278756512246, 16.653666666666663, 16.68105457682108))